There is a great deal of silicate vapor (also referred to as gas) in the moon-forming region of a synestia. Any growing silicate moonlets (also referred to as condensates or liquid droplets) will orbit within the moon-forming region of a synestia and will encounter this gas. The gas will impart a drag upon the moonlets, causing them to slow down and lose angular velocity. The orbit of the moonlets will decay over time in accord with the strength of the gas drag acceleration.

In [ ]:
#INSESRT DIAGRAM HERE

Let's get a sense for the magnitude of gas drag acceleration that we're dealing with. It is assumed that the moonlets experience a simple gas drag. The equation is as follows:
$$a_{drag}=\frac{C_D\ A \ (u_{gas} - u_{body})^2}{2 \ m_{body}}$$

Simple gas drag acceleration a$_{drag}$ is dependent on: the mass of the body that is being dragged by the gas $m_{body}$, the cross sectional area of the body that the gas is interacting with A, the gas drag coefficient C$_D$ (what thermal regime we are in), and the relative velocity between the gas and the body being dragged u$_{gas}$-u$_{body}$. Later, we calculate the gas drag coefficient using the Adachi et al (1976) formulation, but here you can play with its value.

In [6]:
import numpy as np
import math
from ipywidgets import *
import matplotlib.pyplot as plt
v_rel = np.arange(0.0, 300., 1.) #relative velocity between gas and body in m/s
Moon_density = 3300. #lunar density in kg/m3; 3.3 g/cc
def pltdrag(rad_body, C_D):
    mass_body = Moon_density*(4./3.)*math.pi*(rad_body**3) #mass of body in kg; assume spherical body
    a_drag = 0.5*C_D*math.pi*(rad_body**2)*(v_rel**2)/mass_body #acceleration imparted on body due to gas drag in m/s2; assume spherical cross section
    plt.plot(v_rel, a_drag)
    plt.xlabel('velocity of the gas relative to the velocity of the moonlet (m/s)')
    plt.ylabel('gas drag acceleration (m/s$^2$)')
    plt.show()

#PLOT ACCELERATION RELATIVE TO SOMETHING ELSE (g's, roller coasters, astronauts, etc.)    
#RELATIVE SCALE IN GENERAL: how big the moonlets are (marble, golf ball, grain of sand, yoga ball, soccer field, etc.), how fast the relative and absolute velocity is
#MAYBE THIS SHOULD BE PRINTED OUTPUT?    
style = {'description_width': 'initial'}
interact(pltdrag, rad_body=FloatLogSlider(value=1, base=10, min=-3, max=6, step=1, description='Size of moonlet (m)', style=style),
         C_D=FloatSlider(value=2, min=0, max=2.5, step=0.5, description='Gas drag coefficient C$_D$ (-)', continuous_update=False, readout=True, readout_format='.1f', style=style))

interactive(children=(FloatLogSlider(value=1.0, description='Size of moonlet (m)', max=6.0, min=-3.0, step=1.0…

<function __main__.pltdrag(rad_body, C_D)>

As expected, gas drag acceleration is greater when the mass of the moonlet is smaller and the gas drag coefficient and relative velocity are larger.

Now, let's see how a single moonlet travels through the vapor in a synestia. First, we will look at how a moonlet moves on a stable Keplerian orbit. Next, we will see the effects of the second-order term of gravity on the orbit of the moonlet. Moonlets, once formed, will quickly equilibrate to the midplane. So, we will look at orbits in the midplane first. Moonlets condense as droplets at the photosphere (where the vapor is saturated and has a pressure that lies on the vapor dome). We will check orbits of moonlets condensing at the photosphere (starting off the midplane) next. Let's check the decay time for various sizes of moonlets and various starting semi-major axes.

In [15]:
#CALCULATE KEPLERIAN ORBIT WITH NORMAL GRAVITY
G=6.674e-11 #SI
%run synfits.py
Mass_syn=np.sum(SNAP_Canup.m) #kg #Earth mass
def pltKep(x0,y0=0,z0=0,dt=1.):
    #1 counter-clockwise orbit
    #default starts at (x,y,z)=(radius,0,0)
    #with initial velocity (v_x,v_y,v_z)=(0,vel,0)
    #dt is in seconds
    theta=0. #radians
    while (theta < 2*math.pi):
        radius = (x0**2 + y0**2 + z0**2)**0.5 #distance from origin of central body (m)
        accel_grav=-G*Mass_syn/radius**2 #total acceleration (m/s^2)
        vel=np.sqrt(G*Mass_syn/radius) #total velocity (m/s)
        vx0=-vel*y0/(x0**2 + y0**2)**0.5 #x vel. (m/s)
        vy0=vel*x0/(x0**2 + y0**2)**0.5 #y vel. (m/s)
        vz0=vel*z0/radius #z vel. (m/s)
        ax0=accel_grav*vx0/vel #x accel. (m/s^2)
        ay0=accel_grav*vy0/vel #y accel. (m/s^2)
        az0=accel_grav*vz0/vel #z accel. (m/s^2)
        dax=-dt*G*Mass_syn*ax0/(vel*(radius**2)) #change in x accel. (m/s^2)
        day=-dt*G*Mass_syn*ay0/(vel*(radius**2)) #change in y accel. (m/s^2)
        daz=-dt*G*Mass_syn*az0/(vel*(radius**2)) #change in z accel. (m/s^2)
        ax=ax0+dax #update x acceleration (m/s^2)
        ay=ay0+day #update y acceleration (m/s^2)
        az=az0+daz #update z acceleration (m/s^2)
        dvx=ax*dt #change in x vel. (m/s)
        dvy=ay*dt #change in y vel. (m/s)
        dvz=az*dt #change in z vel. (m/s)
        vx=vx0+dvx #update x vel. (m/s)
        vy=vy0+dvy #update y vel. (m/s)
        vz=vz0+dvz #update z vel. (m/s)
        dx=vx*dt #change in x (m)
        dy=vy*dt #change in y (m)
        dz=vz*dt #change in z (m)
        x=x0+dx #position x (m)
        y=y0+dy #position y (m)
        z=z0+dz #position z (m)
        x0=x #update x0 (m)
        y0=y #update y0 (m)
        z0=z #update z0 (m)
        vx0=vx #update vx0 (m/s)
        vy0=vy #update vy0 (m/s)
        vz0=vz #update vz0 (m/s)
        ax0=ax #update ax0 (m/s^2)
        ay0=ay #update ay0 (m/s^2)
        az0=az #update az0 (m/s^2)
        if (x > 0.):
            if (y > 0.):
                theta=math.atan(y/x)
            elif (y < 0.):
                theta=math.atan(y/x)
            else:
                theta=2*math.pi #if zero, then never ending loop
        elif (x < 0.):
            if (y > 0.):
                theta=math.atan(y/x)
            elif (y < 0.):
                theta=math.atan(y/x)
            else:
                theta=math.pi
        else:
            if (y > 0):
                theta=0.5*math.pi
            else:
                theta=1.5*math.pi
        plt.plot(x,y,'.')
        plt.xlabel('x (m)')
        plt.ylabel('y (m)')
        plt.show()
    
interact(pltKep,
         x0=FloatSlider(value=20e6, min=10e6, max=60e6, step=10e6,
                        description='Starting distance from central body (m)',
                        continuous_update=False, readout=True, readout_format='.1f', style=style))  

Read 180233 particles from TE_Example01_Cool05_snapshot_4096_long
Read 201779 particles from TE_Example03_Cool01_snapshot_10500_long
Read 100989 particles from TE_Example07_CoolB01_snapshot_7200_long


interactive(children=(FloatSlider(value=20000000.0, continuous_update=False, description='Starting distance fr…

<function __main__.pltKep(x0, y0=0, z0=0, dt=1.0)>

In [ ]:
#IMPORT GADGET-2 SNAPSHOT (VAPOR IN SYNESTIA)
#FIT THERMAL PARAMETER PROFILES: PRESSURE, TEMPERATURE, DENSITY
#FIT IN-PLANE VELOCITY PROFILE
#CALCULATE ORBITAL DECAY; UPDATE ACCELERATION DUE TO DRAG
widgets.Dropdown(
    options=[('Thickness of a plastic ID card (mm)', 1.e-3),
             ('Diameter of a belly button (cm)', 1.e-2),
             ('Coconut (dm)', 0.1),
             ('Cow (m)', 1.),
             ('230 story building (km)', 1.e3),
             ('Width of the United States (Mm)', 1.e6),
             ('The Moon (3.5 Mm)', 3.5e6)],
    value=1,
    description='Size (diameter) of moonlet:',
)
#need leapfrog integrator

In [ ]:
#REFERENCES